In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Set the seed for reproducibility
np.random.seed(42)

In [3]:
entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
nombre_experimento = 'LSTM-GRU_producto'
#nombre_carpeta_optuna = 'LSTM-GRU_producto'
ventana_input = 12
ventana_output = 2




# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)


dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))


dataset_completo.head()


,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000


In [5]:
ventas_producto_mes = dataset_completo.groupby(['Timestamp', 'product_id'])['tn'].sum().reset_index()
ventas_producto_mes = ventas_producto_mes.rename(columns={'tn': 'y', 'Timestamp': 'ds'})
ventas_producto_mes

,ds,product_id,y
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011
...,...,...,...
22370,2019-12-01,21263,0.01270
22371,2019-12-01,21265,0.05007
22372,2019-12-01,21266,0.05121
22373,2019-12-01,21267,0.01569


In [62]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
import warnings
import tensorflow as tf
from keras import backend as K

# Enable TensorFlow debug mode for better error tracking
tf.data.experimental.enable_debug_mode()

# Filter keras UserWarnings to reduce output noise
warnings.filterwarnings('ignore', category=UserWarning, module='keras')

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Function to add additional features to the data
def add_features(data):
    df = pd.DataFrame(data, columns=['y'])
    df['lag_1'] = df['y'].shift(1)
    df['lag_2'] = df['y'].shift(2)
    df['ma_3'] = df['y'].rolling(window=3).mean()
    df['ma_6'] = df['y'].rolling(window=6).mean()
    df['std_3'] = df['y'].rolling(window=3).std()
    df['trend'] = range(len(df))
    df['month'] = df.index % 12 + 1
    df['sin_month'] = np.sin(2 * np.pi * df['month']/12)
    df['cos_month'] = np.cos(2 * np.pi * df['month']/12)
    df.dropna(inplace=True)
    return df.values

# Custom loss function combining MSE and MAE
def custom_loss(y_true, y_pred):
    mse = K.mean(K.square(y_true - y_pred))
    mae = K.mean(K.abs(y_true - y_pred))
    return 0.5 * mse + 0.5 * mae

# Function to build the LSTM model
def build_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True, input_shape=input_shape)),
        Dropout(0.2),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        GRU(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)
    return model

# Main execution script
if __name__ == "__main__":
    seq_length = 6
    product_ids_lstm = list(range(20001, 20014))
    lista_productos_lstm = []
    lista_predicciones_mes2_lstm = []

    for producto in product_ids_lstm:
        print(f"Analyzing product_id: {producto}")

        # Assuming 'ventas_producto_mes' is your sales data
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()

        # Sorting by date if 'date' column exists, otherwise by index
        if 'date' in ventas_mes_por_producto.columns:
            ventas_mes_por_producto['date'] = pd.to_datetime(ventas_mes_por_producto['date'])
            ventas_mes_por_producto = ventas_mes_por_producto.sort_values('date')
        else:
            ventas_mes_por_producto = ventas_mes_por_producto.sort_index()

        data = ventas_mes_por_producto[['y']].values

        if len(data) < seq_length + 2:
            print(f"Insufficient data for product {producto}. Using mean prediction.")
            mean_prediction = np.mean(data)
            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(mean_prediction)
            continue

        data = np.log1p(data)

        data_with_features = add_features(data)

        scaler = RobustScaler()
        data_scaled = scaler.fit_transform(data_with_features)

        X, y = create_sequences(data_scaled, seq_length)

        print(f"X shape: {X.shape}, y shape: {y.shape}")

        if len(X) < 10:
            print(f"Insufficient data for product {producto} after sequence creation. Using mean prediction.")
            mean_prediction = np.mean(np.expm1(data))
            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(mean_prediction)
            continue

        tscv = TimeSeriesSplit(n_splits=3)
        predictions = []

        for train_index, test_index in tscv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            model = build_model((seq_length, data_with_features.shape[1]))

            early_stop = EarlyStopping(monitor='val_loss', patience=20)
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6)

            model.fit(X_train, y_train, 
                      epochs=300,
                      batch_size=32,
                      validation_split=0.2,
                      callbacks=[early_stop, reduce_lr], 
                      verbose=0)

            pred = model.predict(X_test)
            predictions.extend(pred.flatten())

        try:
            last_sequence = data_scaled[-seq_length:]
            last_sequence = np.expand_dims(last_sequence, axis=0)

            next_month_prediction = model.predict(last_sequence)
            next_month_prediction = scaler.inverse_transform(np.hstack([next_month_prediction, np.zeros((next_month_prediction.shape[0], data_with_features.shape[1]-1))]))[0][0]
            next_month_prediction = np.expm1(next_month_prediction)

            next_month_prediction_scaled = scaler.transform(np.hstack([np.array([[np.log1p(next_month_prediction)]]), np.zeros((1, data_with_features.shape[1]-1))]))
            last_sequence = np.concatenate([last_sequence[:, 1:, :], np.expand_dims(next_month_prediction_scaled, axis=1)], axis=1)
            next_2_month_prediction = model.predict(last_sequence)
            next_2_month_prediction = scaler.inverse_transform(np.hstack([next_2_month_prediction, np.zeros((next_2_month_prediction.shape[0], data_with_features.shape[1]-1))]))[0][0]
            next_2_month_prediction = np.expm1(next_2_month_prediction)

            # Ensemble with historical average
            historical_avg = np.mean(np.expm1(data[-12:]))  # Average of last 12 months
            final_prediction = 0.7 * next_2_month_prediction + 0.3 * historical_avg

            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(final_prediction)
            print(f"Prediction for product {producto}: {final_prediction}")
        except Exception as e:
            print(f"Error during final prediction for product {producto}: {str(e)}")
            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(np.nan)

    # Creating results DataFrame and saving predictions to CSV
    resultados_kaggle_lstm = pd.DataFrame({'product_id': lista_productos_lstm, 'tn': lista_predicciones_mes2_lstm})
    directorio_script = '.'  # Change this to your desired directory
    resultados_kaggle_lstm.to_csv(os.path.join(directorio_script, 'predicciones_lstm_improved8_extended.csv'), index=False)

    # Printing summary of predictions
    print("\nSummary of predictions:")
    for producto, prediccion in zip(lista_productos_lstm, lista_predicciones_mes2_lstm):
        print(f"Product {producto}: {prediccion}")

    # Printing total products analyzed and counts of valid vs. missing predictions
    print(f"\nTotal products analyzed: {len(lista_productos_lstm)}")
    print(f"Products with valid predictions: {sum(~np.isnan(lista_predicciones_mes2_lstm))}")
    print(f"Products without predictions: {sum(np.isnan(lista_predicciones_mes2_lstm))}")


Analyzing product_id: 20001
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 52ms/step
Prediction for product 20001: 1464.2740970017128
Analyzing product_id: 20002
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 49ms/step
Prediction for product 20002: 1043.10414579687
Analyzing product_id: 20003
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 54ms/step
Prediction for product 20003: 1021.4891134540533
Analyzing product_id: 20004
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 54ms/step
Prediction for product 20004: 665.8367518203794
Analyzing product_id: 20005
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 53ms/step
Prediction for product 20005: 558.741504975904
Analyzing product_id: 20006
X shape: (25, 6, 10), y shape: (25,)
1/1 [==============================] - 0s 55ms/step
Prediction for product 20006: 484.8531843054999
Analyzing 

In [23]:
# PROMEDIO PARA RESTO

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 12  # Use last 12 months to predict next month

lista_productos_lstm = []
lista_predicciones_mes2_lstm = []

# Define the range of product_ids to skip
skip_product_ids = list(range(20001, 20014))

for producto in ventas_producto_mes['product_id'].unique():
    if producto in skip_product_ids:
        continue
    
    if producto in productos_predecir['product_id'].values:
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
        data = ventas_mes_por_producto['y'].values.reshape(-1, 1)

        # If not enough data, use mean as prediction
        if len(data) < seq_length + 1:
            mean_prediction = np.mean(data)
            print(f"Not enough data for product {producto}, using mean prediction: {mean_prediction}")
            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(mean_prediction)
            continue

        # Scale data
        scaler = MinMaxScaler()
        data_scaled = scaler.fit_transform(data)

        # Create sequences
        X, y = create_sequences(data_scaled, seq_length)

        # Check if there are enough data points
        if len(X) == 0:
            mean_prediction = np.mean(data)
            print(f"Not enough data for product {producto}, using mean prediction: {mean_prediction}")
            lista_productos_lstm.append(producto)
            lista_predicciones_mes2_lstm.append(mean_prediction)
            continue

        # Split into train and test sets
        split = int(len(X) * 0.8)
        X_train, y_train = X[:split], y[:split]
        X_test, y_test = X[split:], y[split:]

        # Define LSTM model
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        # Train model
        model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1)

        # Make predictions
        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)
        y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Predict next 2 months
        last_sequence = data_scaled[-seq_length:]
        last_sequence = np.expand_dims(last_sequence, axis=0)
        next_month_prediction = model.predict(last_sequence)
        next_month_prediction = scaler.inverse_transform(next_month_prediction)[0][0]

        # Prepare last sequence for the next prediction
        next_month_prediction_scaled = scaler.transform(np.array([[next_month_prediction]]))
        last_sequence = np.append(last_sequence[:, 1:, :], np.expand_dims(next_month_prediction_scaled, axis=0), axis=1)
        next_2_month_prediction = model.predict(last_sequence)
        next_2_month_prediction = scaler.inverse_transform(next_2_month_prediction)[0][0]

        lista_productos_lstm.append(producto)
        lista_predicciones_mes2_lstm.append(next_2_month_prediction)

# Save results
resultados_kaggle_lstm = pd.DataFrame({'product_id': lista_productos_lstm, 'tn': lista_predicciones_mes2_lstm})
resultados_kaggle_lstm.to_csv(os.path.join(directorio_script, 'predicciones_lstm64.csv'), index=False)


Epoch 1/20
19/19 [==============================] - 1s 62ms/step - loss: 0.0936
Epoch 2/20
19/19 [==============================] - 1s 59ms/step - loss: 0.0525
Epoch 3/20
19/19 [==============================] - 1s 59ms/step - loss: 0.0472
Epoch 4/20
19/19 [==============================] - 1s 57ms/step - loss: 0.0476
Epoch 5/20
19/19 [==============================] - 1s 58ms/step - loss: 0.0481
Epoch 6/20
19/19 [==============================] - 1s 58ms/step - loss: 0.0493
Epoch 7/20
19/19 [==============================] - 1s 62ms/step - loss: 0.0574
Epoch 8/20
19/19 [==============================] - 1s 59ms/step - loss: 0.0602
Epoch 9/20
19/19 [==============================] - 1s 58ms/step - loss: 0.0456
Epoch 10/20
19/19 [==============================] - 1s 58ms/step - loss: 0.0457
Epoch 11/20
19/19 [==============================] - 1s 59ms/step - loss: 0.0450
Epoch 12/20
19/19 [==============================] - 1s 58ms/step - loss: 0.0459
Epoch 13/20
19/19 [==================

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Cargar los archivos CSV
file1 = 'predicciones_lstm_improved8_extended.csv'
file2 = 'predicciones_lstm64.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Concatenar los DataFrames
df_concat = pd.concat([df1, df2], ignore_index=True)

# Guardar el DataFrame fusionado en un nuevo archivo CSV
output_file = 'predicciones_lstm_fusionadas.csv'
df_concat.to_csv(output_file, index=False)

print(f"Se han fusionado y guardado las predicciones en {output_file}.")
